In [1]:
import os
import csv
from pprint import pprint
import pandas as pd
import pickle
import collections

In [2]:
hashPath = "/Users/Bya/git/EPLdata/"
dataPath = "/Users/Bya/Dropbox/Research/datas/"
modelPath = "/Users/Bya/Dropbox/Research/datas/Pickles/Models/"
tokenizerPath = "/Users/Bya/git/predictEPL/Tokenizers/"


# Custom Library
os.chdir(tokenizerPath)
import sentiment_aware as sa
os.chdir(hashPath)
from config import Hashtags
os.chdir('/Users/Bya/git/predictEPL/')
from MyFunctions import dataIO
from MyFunctions import plotPie

In [3]:
model = "MovieShortReview"

# read saved classifier
saved_classifier_f = open(modelPath + model + "/naiveBayes.pickle", "rb")
saved_classifier = pickle.load(saved_classifier_f)
saved_classifier_f.close()

# read saved word features
word_features5k_f = open(modelPath + model + "/word_features5k.pickle", "rb")
word_features = pickle.load(word_features5k_f)
word_features5k_f.close()

# Define Sentiment Tokenizer Class as tok
tok = sa.Tokenizer(preserve_case=False)
def find_features(document):
    words = tok.tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features

# Define Sentiment Analysis function. Using aboves
def sentiment(text):
    return saved_classifier.classify(find_features(text))

In [4]:
def SentimentUsersLastTweet(GW, home_team, away_team):
    filePath = '/Users/Bya/Dropbox/Research/datas/Tweet2Sides/' + GW +'/'
    dfHA = dataIO.CSVtoDF(filePath + home_team + '_' + away_team + '.csv', ['date', 'text', 'user', 'tags', 'side'])

    dic_users_last_tweet = {}

    for row in dfHA.iterrows():
        user = row[1]['user']
        side = row[1]['side']
        text = row[1]['text']

        dic_users_last_tweet[user] = (side, text)

    list_home_users_name_last = filter(lambda key: dic_users_last_tweet[key][0] == 'home', dic_users_last_tweet)
    list_away_users_name_last = filter(lambda key: dic_users_last_tweet[key][0] == 'away', dic_users_last_tweet)

    list_home_text = map(lambda user: dic_users_last_tweet[user][1], list_home_users_name_last)
    list_away_text = map(lambda user: dic_users_last_tweet[user][1], list_away_users_name_last)
    
    home_sent = map(lambda tweet: sentiment(tweet), list_home_text)
    away_sent = map(lambda tweet: sentiment(tweet), list_away_text)
    
    listInfo = [home_team, away_team,
                collections.Counter(home_sent)['pos'], 
                collections.Counter(home_sent)['neg'],
                collections.Counter(away_sent)['pos'],
                collections.Counter(away_sent)['neg']]
    dataIO.ListSaveToCSVappend(listInfo, GW + '_UsersLastTweet', '/Users/Bya/Dropbox/Research/datas/Results/')

In [8]:
listGamesW6 = [
#                ('Chelsea', 'Arsenal'),
               
#                ('Villa', 'WestBromwich'),
#                ('Bournemouth', 'Sunderland'),
#                ('Newcastle', 'Watford'),
#                ('Stoke', 'Leicester'),
#                ('Swansea', 'Everton'),
               
#                ('City', 'WestHam'),
               
               ('Tottenham', 'Crystal'),
               
               ('Liverpool', 'Norwich'),
               ('Southampton', 'United')]


GW = 'GW6'
for i in range(len(listGamesGW5)):
    SentimentUsersLastTweet(GW, listGamesW6[i][0], listGamesW6[i][1])

IndexError: list index out of range